In [ ]:
%pip install bs4

In [9]:
from datetime import datetime

def convertir_date(date_string):
    try:
        # Convertir en objet datetime
        date_obj = datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S.%f%z')
        
        # Extraire la date au format souhaité (aaaa-mm-jj)
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    except ValueError:
        return "Format de date invalide. Assurez-vous que la chaîne est au format 'aaaa-mm-jjThh:mm:ss.ssssss+hh:mm'."

# Exemple d'utilisation
date_string = '2021-03-26T10:55:53.379470+01:00'
resultat = convertir_date(date_string)
print(resultat)


<class 'str'>


In [92]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
# URL de l'API
api_url = "https://biodiv-sports.fr/api/v2/sensitivearea/"


from geopy.geocoders import Nominatim
import time


geolocator = Nominatim(user_agent="myApplication")

def get_location_info(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    region = address.get('state', '')
    country = address.get('country', '')
    departement = address.get('county', '')
    return  region, departement, country


def convertir_date(date_string):
    try:
        # Convertir en objet datetime
        date_obj = datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S.%f%z')
        
        # Extraire la date au format souhaité (aaaa-mm-jj)
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    except ValueError:
        return "Format de date invalide. Assurez-vous que la chaîne est au format 'aaaa-mm-jjThh:mm:ss.ssssss+hh:mm'."

def add_months_names(df):
    # Ajouter le nom du mois
    months_list = ['janvier', 'fevrier', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'decembre']
    df_with_months = df.assign(
        months=df['period'].apply(lambda x: [months_list[i] for i, value in enumerate(x) if value == 1])
    )
    return df_with_months


def remove_html_tags(text):
    # Obtenez le texte sans balises HTML
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    
    return cleaned_text 


def data_extract(api_url):
    # Récupération des données depuis l'API
    response = requests.get(api_url)
    res = response.json()
    res = res["results"]
    return res

import pandas as pd

def data_transformation(res: list) -> pd.DataFrame:
    # Préparation des données pour la création du DataFrame
    data = [{
        "create_datetime": convertir_date(item["create_datetime"]),
        "id": item["id"],
        "description": remove_html_tags(item["description"]["fr"]),
        "name": item["name"]["fr"],
        "structure": item["structure"],
        "elevation": item["elevation"],
        "provider": item["provider"],
        "coordonnees": item["geometry"]["coordinates"][0][0][0] if item["geometry"]["type"] == "MultiPolygon" else item["geometry"]["coordinates"][0][0],
        "period": [int(bool_) for bool_ in item["period"]] if isinstance(item["period"], list) else item["period"],
        "update_datetime": convertir_date(item["update_datetime"])
    } for item in res]

    # Création du DataFrame
    df = pd.DataFrame(data)
    df[["region", "departement", "Pays"]] = list(df["coordonnees"].apply(lambda raw: get_location_info(raw[1], raw[0])))
    df = add_months_names(df)
    df = df.explode("months")
    df.reset_index(inplace=True)
    df["create_datetime"] = pd.to_datetime(df["create_datetime"])
    df.drop(["period", "coordonnees"], axis=1, inplace=True)
    return df

  
res = data_extract(api_url)

data = data_transformation(res)
#
data


,index,create_datetime,id,description,name,structure,elevation,provider,update_datetime,region,departement,Pays,months
0,0,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,janvier
1,0,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,fevrier
2,0,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,mars
3,0,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,avril
4,0,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,mai


In [19]:
from geopy.geocoders import Nominatim
import time

# Create a geolocator object with a specific user_agent
geolocator = Nominatim(user_agent="myApplication")

def get_location_info(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    region = address.get('state', '')
    country = address.get('country', '')
    departement = address.get('county', '')
    return  region, country, departement

# Example usage
latitude =  44.3727858
                           
longitude =  6.8592967


state, country,county = get_location_info(latitude, longitude)

print(f"Region: {state}, departement: {county} country: {country}")


"Region: Provence-Alpes-Côte d'Azur, departement: Alpes-de-Haute-Provence country: France"

In [10]:
import pandas as pd

def add_months_names(df):
    months_list = ['janvier', 'fevrier', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'decembre']
    df_with_months = df.assign(
        months=df['period'].apply(lambda x: [months_list[i] for i, value in enumerate(x) if value == 1])
    )
    return df_with_months


In [4]:
import pandas as pd
from datetime import datetime, timedelta

def ajust_create_datetime(df, target="create_datetime", Groupby="index"):
    base_date = df[target].iloc[0]
    df["diff"] = ((df.groupby(Groupby).cumcount() * pd.DateOffset(months=1)))
    df["transformed_date"] = df[target] + df["diff"]
    return df

In [114]:
import time
from datetime import datetime
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

# URL de l'API
API_URL = "https://biodiv-sports.fr/api/v2/sensitivearea/"

geolocator = Nominatim(user_agent="myApplication")

def get_location_info(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    region = address.get('state', '')
    country = address.get('country', '')
    departement = address.get('county', '')
    return region, departement, country

def type_conversion(df):
    str_cols = ["id", "description", "name", "structure", "region", "departement", "Pays"]
    date_cols = ["update_datetime", "create_datetime"]

    df[str_cols] = df[str_cols].apply(lambda col: col.astype(str))
    df[date_cols] = df[date_cols].apply(lambda col: pd.to_datetime(col))

    return df

def convertir_date(date_string):
    try:
        date_obj = datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S.%f%z')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    except ValueError:
        return "Format de date invalide. Assurez-vous que la chaîne est au format 'aaaa-mm-jjThh:mm:ss.ssssss+hh:mm'."

def add_months_names(df):
    months_list = ['janvier', 'fevrier', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'decembre']
    df_with_months = df.assign(
        months=df['period'].apply(lambda x: [months_list[i] for i, value in enumerate(x) if value == 1])
    )
    return df_with_months

def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    return cleaned_text 

def data_extract(api_url):
    response = requests.get(api_url)
    res = response.json()
    res = res["results"]
    return res

def data_transformation(res):
    data = [{
        "create_datetime": convertir_date(item["create_datetime"]),
        "id": item["id"],
        "description": remove_html_tags(item["description"]["fr"]),
        "name": item["name"]["fr"],
        "structure": item["structure"],
        "species_id": item["species_id"],
        "practices": item["practices"],
        "coordonnees": item["geometry"]["coordinates"][0][0][0] if item["geometry"]["type"] == "MultiPolygon" else item["geometry"]["coordinates"][0][0],
        "period": [int(bool_) for bool_ in item["period"]] if isinstance(item["period"], list) else item["period"],
        "update_datetime": convertir_date(item["update_datetime"])
    } for item in res]

    df = pd.DataFrame(data)
    df[["region", "departement", "Pays"]] = list(df["coordonnees"].apply(lambda raw: get_location_info(raw[1], raw[0])))
    df = add_months_names(df)
    df = df.explode("months")
    df = df.explode("practices")
    df.reset_index(inplace=True)
    df = type_conversion(df)
    df.drop(["period", "coordonnees","index"], axis=1, inplace=True)
    return df

res = data_extract(API_URL)
data = data_transformation(res)

conn = sqlite3.connect("biodiv_sports.db")
cursor = conn.cursor()

table_schema = '''CREATE TABLE IF NOT EXISTS sensitive_areas (
                       id TEXT PRIMARY KEY,
                       description TEXT,
                       name TEXT,
                       structure TEXT,
                       species_id INTEGER,
                       practices TEXT,
                       create_datetime TIMESTAMP,
                       update_datetime TIMESTAMP,
                       région TEXT,
                       département TEXT,
                       pays TEXT,
                       mois TEXT);'''

cursor.execute(table_schema)
conn.commit()

processed_data = data.copy()
processed_data.to_sql(name='sensitive_areas', con=conn, if_exists='replace', index=False)
conn.close()


,create_datetime,id,description,name,structure,species_id,practices,update_datetime,region,departement,Pays,months
0,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,3,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,janvier
1,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,5,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,janvier
2,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,6,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,janvier
3,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,1,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,janvier
4,2021-03-26,997,Un Parc national est un territoire reconnu com...,Coeur du Parc national du Mercantour,Parc National du Mercantour,NaN,2,2022-03-22,Provence-Alpes-Côte d'Azur,Alpes-de-Haute-Provence,France,janvier
...,...,...,...,...,...,...,...,...,...,...,...,...
1887,2017-12-11,240,Nidification de l'Aigle royalLes pratiques qui...,Aigle royal,LPO,1.0,7,2018-03-13,Auvergne-Rhône-Alpes,Drôme,France,juillet
1888,2017-12-11,240,Nidification de l'Aigle royalLes pratiques qui...,Aigle royal,LPO,1.0,2,2018-03-13,Auvergne-Rhône-Alpes,Drôme,France,juillet
1889,2017-12-11,240,Nidification de l'Aigle royalLes pratiques qui...,Aigle royal,LPO,1.0,3,2018-03-13,Auvergne-Rhône-Alpes,Drôme,France,août
1890,2017-12-11,240,Nidification de l'Aigle royalLes pratiques qui...,Aigle royal,LPO,1.0,7,2018-03-13,Auvergne-Rhône-Alpes,Drôme,France,août


In [116]:
import sqlite3
conn = sqlite3.connect("biodiv_sports.db")
cursor = conn.cursor()

table_schema = '''CREATE TABLE IF NOT EXISTS sensitive_areas (
                       id TEXT PRIMARY KEY,
                       description TEXT,
                       name TEXT,
                       structure TEXT,
                       species_id INTEGER,
                       practices TEXT,
                       create_datetime TIMESTAMP,
                       update_datetime TIMESTAMP,
                       région TEXT,
                       département TEXT,
                       pays TEXT,
                       mois TEXT);'''

cursor.execute(table_schema)
conn.commit()

processed_data = data.copy()
processed_data.to_sql(name='sensitive_areas', con=conn, if_exists='replace', index=False)
conn.close()

In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892 entries, 0 to 1891
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   create_datetime  1892 non-null   datetime64[ns]
 1   id               1892 non-null   object        
 2   description      1892 non-null   object        
 3   name             1892 non-null   object        
 4   structure        1892 non-null   object        
 5   species_id       308 non-null    float64       
 6   practices        1892 non-null   object        
 7   update_datetime  1892 non-null   datetime64[ns]
 8   region           1892 non-null   object        
 9   departement      1892 non-null   object        
 10  Pays             1892 non-null   object        
 11  months           1892 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 177.5+ KB


In [109]:
data["provider"].unique()

array([''], dtype=object)

In [97]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from datetime import datetime

# Set up API url and user agent
API_URL = "https://biodiv-sports.fr/api/v2/sensitivearea/"
USER_AGENT = "My Application"

# Initialize Nominatim with a custom user agent to comply with usage policy
geolocator = Nominatim(user_agent=USER_AGENT)

def extract_location_info(latitude, longitude):
    """Get detailed location information from latitude and longitude"""
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    region = address.get('state', '')
    county = address.get('county', '')
    country = address.get('country', '')
    return region, county, country

def parse_and_format_date(date_string):
    """Convert string into a formatted date object or raise an exception on failure"""
    try:
        date_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%f%z")
        return date_object.strftime("%Y-%m-%d")
    except ValueError:
        return "Invalid date format. Ensure the input string matches '%Y-%m-%dT%H:%M:%S.%f%z'."

def add_month_names(df):
    """Add month names based on period values within each row"""
    months_list = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']

    def filter_valid_entries(series):
        """Filter out series without at least one True value"""
        mask = series != 0
        return series[mask].any()

    filtered_df = df[df['period'].apply(filter_valid_entries)]

    df_with_months = filtered_df.copy().assign(
        moi=[months_list[i] for i, value in enumerate(filtered_df['period']) if value == 1]
    )

    return df_with_months.reindex(df.index)

def clean_html_tags(text):
    """Remove HTML tags from text content"""
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def fetch_data():
    """Fetch sensitive area data using the provided API endpoint."""
    response = requests.get(API_URL)
    response.raise_for_status()
    result = response.json()
    return result["results"]

def transform_data(data):
    """Transform fetched data into a usable Pandas DataFrame"""
    transformed_data = []
    for entry in data:
        coordinates = entry["geometry"]["coordinates"][0][0][0] if entry["geometry"]["type"] == "MultiPolygon" else entry["geometry"]["coordinates"][0][0]
        new_entry = {
            "create_datetime": parse_and_format_date(entry["create_datetime"]),
            "id": entry["id"],
            "description": clean_html_tags(entry["description"]["fr"]),
            "name": entry["name"]["fr"],
            "structure": entry["structure"],
            "elevation": entry["elevation"],
            "provider": entry["provider"],
            "coordinate": coordinates,
            "period": [int(value) for value in entry["period"]] if isinstance(entry["period"], list) else entry["period"],
            "update_datetime": parse_and_format_date(entry["update_datetime"])
        }
        transformed_data.append(new_entry)

    df = pd.DataFrame(transformed_data).sort_values("create_datetime")
    df[["region", "county", "country"]] = list(df["coordinate"].apply(lambda raw : extract_location_info(latitude=raw[1], longitude=raw[0])))
    df = add_month_names(df)
    df = df.explode("mois").reset_index(drop=True)
    df["create_datetime"] = pd.to_datetime(df["create_datetime"])
    df = df.drop(columns=["period", "coordinate"])
    return df

if __name__ == "__main__":
    data = fetch_data()
    transformed_dataframe = transform_data(data)
    print(transformed_dataframe.head())

AttributeError: 'int' object has no attribute 'any'